In [3]:
from bs4 import BeautifulSoup
import requests
from tqdm.autonotebook import tqdm

/tmp/ipykernel_26942/3529437905.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
BASE_URL = "https://www.desmog.com/climate-disinformation-database/"
response = requests.get(BASE_URL)
soup = BeautifulSoup(response.content, 'html.parser')

element_list = soup.find("div", class_="display-view-entries grid-view-entries active")

results = []
for x in element_list:
    link = x.find("a")["href"]
    name = x.find("div", class_="grid-view-entry-name entry-name").text.strip()
    country = x.find("div", class_="grid-view-entry-country").text.strip()
    results.append((name, country, link))

In [49]:
def get_article(link:str):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')

    content = soup.find("div", class_="elementor-widget-wrap elementor-element-populated")
    
    divs = content.find_all("div", class_="elementor-widget-container")

    for div in divs:
        if div.find("p"):
            content= div
            break

    sections = {}
    current_section = None
    paragraphs = []
    for element in soup.find_all(["h1", "h2", "p"]):
        if element.name in ["h1", "h2"]:
            # If there's an ongoing section, save it to the dictionary
            if current_section and paragraphs:
                sections[current_section] = paragraphs
                paragraphs = []  # Reset paragraphs for the new section

            # Set the new section title as the current section
            current_section = element.get_text(strip=True)
        elif element.name == "p":
            # Add paragraph text to the current section
            paragraphs.append(element.get_text(strip=True))

    # Add the last section to the dictionary (if it has any paragraphs)
    if current_section and paragraphs:
        sections[current_section] = paragraphs


    blockquotes = content.find_all("blockquote")
    quotes = []
    for quote in blockquotes: 
        text = " ".join([p.text[1:-1] for p in quote.find_all("p")])
        text = text.split("”")[0].strip()
        quotes.append(text.strip())
    sections["quotes"] = quotes
    
    return sections


In [50]:
sections = get_article("https://www.desmog.com/john-stuart-agnew/")

In [58]:
import pandas as pd
df = pd.DataFrame(results, columns =['Name', 'Country', 'Link'])

In [59]:
df

,Name,Country,Link
0,55 Tufton Street,United Kingdom,https://www.desmog.com/55-tufton-street/
1,60 Plus Association,United States,https://www.desmog.com/60-plus-association/
2,Tony Abbott,Australia,https://www.desmog.com/tony-abbott/
3,Accuracy in Media,United States,https://www.desmog.com/accuracy-media/
4,Acton Institute,"United States, Italy",https://www.desmog.com/acton-institute/
...,...,...,...
918,Karl Zeller,United States,https://www.desmog.com/karl-zeller/
919,Heather Zichal,United States,https://www.desmog.com/heather-zichal/
920,Antonino Zichichi,Italy,https://www.desmog.com/antonino-zichichi/
921,Ryan Zinke,United States,https://www.desmog.com/ryan-zinke/


In [ ]:
full_res = []
for index, persona in enumerate(results):
    result = get_article(persona[2])

    stance = result.get("Stance on Climate Change", "")
    key_quotes = result.get("key_quotes", "")
    quotes = result.get("quotes", "")
    full_res = (result[0], result[1], result[2], quotes, stance, key_quotes)

In [65]:
for index, persona in enumerate(results):
    result = get_article(persona[2])
    stance = result.get("Stance on Climate Change", "")
    key_quotes = result.get("key_quotes", "")
    quotes = result.get("quotes", "")

    df.iloc[index]["stance"] = stance
    df.iloc[index]["key_quotes"] = key_quotes
    df.iloc[index]["quotes"] = quotes


/tmp/ipykernel_26942/1101658051.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[index]["stance"] = stance
/tmp/ipykernel_26942/1101658051.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[index]["key_quotes"] = key_quotes
/tmp/ipykernel_26942/1101658051.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[index]["quotes"] = quotes


KeyboardInterrupt: 